`preprocessing_notebook_part_1.ipynb`: outputs a structured parquet format and saves it.

In [2]:
import sys
sys.path.insert(0, '../scripts')

from readparquet import read_parquet
sdf = read_parquet('../data/raw/fhvhv_tripdata_2022-01.parquet')
# convert to pandas dataframe for visualisation
df = sdf.sample(0.001, seed=0).toPandas()

22/07/28 19:31:48 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/07/28 19:31:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/28 19:31:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df.info()
df.sample(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     14731 non-null  object        
 1   dispatching_base_num  14731 non-null  object        
 2   originating_base_num  10793 non-null  object        
 3   request_datetime      14731 non-null  datetime64[ns]
 4   on_scene_datetime     10795 non-null  datetime64[ns]
 5   pickup_datetime       14731 non-null  datetime64[ns]
 6   dropoff_datetime      14731 non-null  datetime64[ns]
 7   PULocationID          14731 non-null  int64         
 8   DOLocationID          14731 non-null  int64         
 9   trip_miles            14731 non-null  float64       
 10  trip_time             14731 non-null  int64         
 11  base_passenger_fare   14731 non-null  float64       
 12  tolls                 14731 non-null  float64       
 13  bcf             

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
6010,HV0003,B03404,B03404,2022-01-15 06:16:56,2022-01-15 06:18:11,2022-01-15 06:20:11,2022-01-15 06:30:17,7,260,1.430,...,0.79,0.00,0.0,0.0,6.65,N,N,,N,N
6774,HV0003,B03404,B03404,2022-01-16 10:09:52,2022-01-16 10:12:10,2022-01-16 10:12:26,2022-01-16 10:25:55,181,17,2.310,...,1.19,0.00,0.0,0.0,10.05,N,N,,N,N
7,HV0005,B03406,None,2022-01-01 11:32:27,NaT,2022-01-01 11:38:48,2022-01-01 11:57:27,68,112,4.983,...,2.64,2.75,0.0,0.0,15.66,N,N,N,N,N
11134,HV0003,B03404,B03404,2022-01-24 22:00:52,2022-01-24 22:03:00,2022-01-24 22:03:16,2022-01-24 22:20:19,96,98,6.880,...,0.00,0.00,0.0,0.0,16.15,N,N,,N,N
3761,HV0003,B03404,B03404,2022-01-10 04:14:09,2022-01-10 04:16:49,2022-01-10 04:17:15,2022-01-10 04:24:19,36,198,1.320,...,0.69,0.00,0.0,0.0,7.21,N,N,,N,N


Hmmmm, it seems like there are too many features necessary for data analysis. Let's remove the ones I am certain are completely useless.

In [5]:
sdf.drop("dispatching_base_num")

hvfhs_license_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0003,B03404,2022-01-01 11:05:31,2022-01-01 11:05:40,2022-01-01 11:07:24,2022-01-01 11:18:28,170,161,1.18,664,24.9,0.0,0.75,2.21,2.75,0.0,0.0,23.03,N,N,,N,N
HV0003,B03404,2022-01-01 11:19:27,2022-01-01 11:22:08,2022-01-01 11:22:32,2022-01-01 11:30:12,237,161,0.82,460,11.97,0.0,0.36,1.06,2.75,0.0,0.0,12.32,N,N,,N,N
HV0003,B03404,2022-01-01 11:43:53,2022-01-01 11:57:37,2022-01-01 11:57:37,2022-01-01 12:07:32,237,161,1.18,595,29.82,0.0,0.89,2.65,2.75,0.0,0.0,23.3,N,N,,N,N
HV0003,B03404,2022-01-01 11:15:36,2022-01-01 11:17:08,2022-01-01 11:18:02,2022-01-01 11:23:05,262,229,1.65,303,7.91,0.0,0.24,0.7,2.75,0.0,0.0,6.3,N,N,,N,N
HV0003,B03404,2022-01-01 11:25:45,2022-01-01 11:26:01,2022-01-01 11:28:01,2022-01-01 11:35:42,229,141,1.65,461,9.44,0.0,0.28,0.84,2.75,0.0,0.0,7.44,N,N,,N,N
HV0003,B03404,2022-01-01 11:34:44,2022-01-01 11:36:52,2022-01-01 11:38:50,2022-01-01 11:51:32,263,79,4.51,762,17.67,0.0,0.53,1.57,2.75,0.0,0.0,12.25,N,N,,N,N
HV0003,B03404,2022-01-01 11:47:51,2022-01-01 11:52:00,2022-01-01 11:53:25,2022-01-01 12:08:56,113,140,3.68,931,16.68,0.0,0.5,1.48,2.75,0.0,0.0,12.75,N,N,,N,N
HV0003,B03404,2022-01-01 11:06:21,2022-01-01 11:06:58,2022-01-01 11:08:58,2022-01-01 11:23:01,151,75,2.77,843,14.41,0.0,0.43,1.28,0.0,0.0,4.0,11.47,N,N,,N,N
HV0003,B03404,2022-01-01 11:27:54,2022-01-01 11:30:26,2022-01-01 11:32:25,2022-01-01 11:44:15,263,229,2.04,710,10.64,0.0,0.32,0.94,2.75,0.0,0.0,9.55,N,N,,N,N
HV0003,B03404,2022-01-01 11:44:59,2022-01-01 11:48:23,2022-01-01 11:50:23,2022-01-01 12:15:30,237,169,8.79,1507,107.56,0.0,0.83,2.45,2.75,0.0,0.0,23.67,N,N,,N,N
